In [12]:
import pandas as pd
from swmm_api.input_file import read_inp_file
from swmm_api.input_file.section_labels import *
from swmm_api.input_file.sections import *
from swmm_api.input_file.helpers import *
from swmm_api.input_file.sections.others import TimeseriesData
from swmm_api.input_file.section_labels import TIMESERIES
import os
os.environ['CONDA_DLL_SEARCH_MODIFICATION_ENABLE']="1"
import numpy as np
import shutil
import yaml
from pyswmm import Simulation,Links,Nodes,RainGages,SystemStats
import matplotlib.pyplot as plt

config = yaml.load(open("./states_yaml/chaohu.yaml"), yaml.FullLoader)
import step_sim
import GI_setting as gs

params={
    'kf':1,
    'kc':1,
}

### 获取建筑屋顶面积，设置GI，保存inp
def add_GI(inp):
    roof=pd.read_csv('polygon.csv')
    unit_area=10
    for i in range(roof.shape[0]):
        number=int(0.5*roof[['roofarea']].iloc[i][0]/unit_area)
        #print(roof[['Name']].iloc[i][0],number)
        # added line of LID_USAGE
        item=gs.LID_setting(roof[['Name']].iloc[i][0],'GreenRoof',number,10,0,0,0,0,'*','*',0)
        inp['LID_USAGE'][roof[['Name']].iloc[i][0]]=item

# S1: RSH

In [13]:
raine = np.load('./RSH/east.npy').tolist()
rainw = np.load('./RSH/west.npy').tolist()

for rainid in range(6):
    inp1 = read_inp_file('./S12/chaohu_.inp')
    inp1[TIMESERIES]['rainfalle']=TimeseriesData('rainfalle',raine[rainid])
    inp1[TIMESERIES]['rainfallw']=TimeseriesData('rainfallw',rainw[rainid])
    inp1.write_file('./S12/Results/RSH/chaohu_rain_'+str(rainid)+'.inp')
    step_sim.simfile('./S12/Results/RSH/',rainid,config,params)


# S2: R

In [14]:
raine = np.load('./RealRain/real.npy').tolist()
rainw = np.load('./RealRain/real.npy').tolist()

for rainid in range(len(raine)):
    inp1 = read_inp_file('./S12/chaohu_.inp') 
    inp1[TIMESERIES]['rainfalle']=TimeseriesData('rainfalle',raine[rainid])
    inp1[TIMESERIES]['rainfallw']=TimeseriesData('rainfallw',rainw[rainid])
    inp1.write_file('./S12/Results/RR/chaohu_rain_'+str(rainid)+'.inp')
    step_sim.simfile('./S12/Results/RR/',rainid,config,params)

# S3: RSH-GR

In [15]:
raine = np.load('./RSH/east.npy').tolist()
rainw = np.load('./RSH/west.npy').tolist()

for rainid in range(6):
    inp = read_inp_file('./S34_GR/chaohu_GI.inp')
    # GI
    #add_GI(inp)
    # rain
    inp[TIMESERIES]['rainfalle']=TimeseriesData('rainfalle',raine[rainid])
    inp[TIMESERIES]['rainfallw']=TimeseriesData('rainfallw',rainw[rainid])
    inp.write_file('./S34_GR/Results/RSH/chaohu_rain_'+str(rainid)+'.inp')
    step_sim.simfile('./S34_GR/Results/RSH/',rainid,config,params)

# S4: R-GR

In [17]:
raine = np.load('./RealRain/real.npy').tolist()
rainw = np.load('./RealRain/real.npy').tolist()

for rainid in range(len(raine)):
    inp = read_inp_file('./S34_GR/chaohu_GI.inp')
    # GI
    #add_GI(inp)
    # rain
    inp[TIMESERIES]['rainfalle']=TimeseriesData('rainfalle',raine[rainid])
    inp[TIMESERIES]['rainfallw']=TimeseriesData('rainfallw',rainw[rainid])
    inp.write_file('./S34_GR/Results/RR/chaohu_rain_'+str(rainid)+'.inp')
    step_sim.simfile('./S34_GR/Results/RR/',rainid,config,params)
    

# S5: RSH-HC

In [6]:
raine = np.load('./RSH/east.npy').tolist()
rainw = np.load('./RSH/west.npy').tolist()

for rainid in range(6):
    inp1 = read_inp_file('./S56_HC/chaohu_RTC.inp')
    inp1[TIMESERIES]['rainfalle']=TimeseriesData('rainfalle',raine[rainid])
    inp1[TIMESERIES]['rainfallw']=TimeseriesData(' rainfallw',rainw[rainid])
    inp1.write_file('./S56_HC/Results/RSH/chaohu_rain_'+str(rainid)+'.inp')
    step_sim.simfile('./S56_HC/Results/RSH/'+str(rainid),'./S56_HC/Results/RSH/chaohu_rain_'+str(rainid)+'.inp',rainid,config,params)


./S56_HC/Results/RSH/0
./S56_HC/Results/RSH/1
./S56_HC/Results/RSH/2
./S56_HC/Results/RSH/3
./S56_HC/Results/RSH/4
./S56_HC/Results/RSH/5


# S6: R-HC

In [7]:
raine = np.load('./RealRain/real.npy').tolist()
rainw = np.load('./RealRain/real.npy').tolist()

for rainid in range(len(raine)):
    inp1 = read_inp_file('./S56_HC/chaohu_RTC.inp')  
    inp1[TIMESERIES]['rainfalle']=TimeseriesData('rainfalle',raine[rainid])
    inp1[TIMESERIES]['rainfallw']=TimeseriesData('rainfallw',rainw[rainid])
    inp1.write_file('./S56_HC/Results/RR/chaohu_rain_'+str(rainid)+'.inp')
    step_sim.simfile('./S56_HC/Results/RR/'+str(rainid),'./S56_HC/Results/RR/chaohu_rain_'+str(rainid)+'.inp',rainid,config,params)

./S56_HC/Results/RR/0
./S56_HC/Results/RR/1
./S56_HC/Results/RR/2
./S56_HC/Results/RR/3


# S7: RSH-GR-HC

In [8]:
raine = np.load('./RSH/east.npy').tolist()
rainw = np.load('./RSH/west.npy').tolist()

for rainid in range(6):
    inp = read_inp_file('./S78_GRHC/chaohu_GI_RTC.inp')
    # GI
    #add_GI(inp)
    # rain
    inp[TIMESERIES]['rainfalle']=TimeseriesData('rainfalle',raine[rainid])
    inp[TIMESERIES]['rainfallw']=TimeseriesData('rainfallw',rainw[rainid])
    inp.write_file('./S78_GRHC/Results/RSH/chaohu_rain_'+str(rainid)+'.inp')
    step_sim.simfile('./S78_GRHC/Results/RSH/'+str(rainid),'./S78_GRHC/Results/RSH/chaohu_rain_'+str(rainid)+'.inp',rainid,config,params)

./S78_GRHC/Results/RSH/0
./S78_GRHC/Results/RSH/1
./S78_GRHC/Results/RSH/2
./S78_GRHC/Results/RSH/3
./S78_GRHC/Results/RSH/4
./S78_GRHC/Results/RSH/5


# S8: R-GR-HC

In [9]:
raine = np.load('./RealRain/real.npy').tolist()
rainw = np.load('./RealRain/real.npy').tolist()

for rainid in range(len(raine)):
    inp = read_inp_file('./S78_GRHC/chaohu_GI_RTC.inp')
    # GI
    #add_GI(inp)
    # rain
    inp[TIMESERIES]['rainfalle']=TimeseriesData('rainfalle',raine[rainid])
    inp[TIMESERIES]['rainfallw']=TimeseriesData('rainfallw',rainw[rainid])
    inp.write_file('./S78_GRHC/Results/RR/chaohu_rain_'+str(rainid)+'.inp')
    step_sim.simfile('./S78_GRHC/Results/RR/'+str(rainid),'./S78_GRHC/Results/RR/chaohu_rain_'+str(rainid)+'.inp',rainid,config,params)
    

./S78_GRHC/Results/RR/0
./S78_GRHC/Results/RR/1
./S78_GRHC/Results/RR/2
./S78_GRHC/Results/RR/3
